# Task 1
### Method (b) - Dense Static Representation

First we import all the libraries we want for this method, and import the training and test csv's.

In [63]:
from gensim.models import Word2Vec
import numpy as np

import pandas as pd
import plotly.graph_objects as go

import re
from nltk import sent_tokenize
from nltk import word_tokenize
from nltk import WordNetLemmatizer
from nltk.corpus import stopwords

from tqdm import tqdm

train_df = pd.read_csv('./Training-dataset.csv')
test_df = pd.read_csv('./Task-1-validation-dataset.csv', header=None)

Now let's clean up the training dataset's synopses to have no punctuation, tags, numbers or special characters.
This will get us an array of words.

In [64]:
def preprocess_and_return_sentences(synopses):
    cleaned_sentences = []

    for synopsis in synopses:
        sentences = re.split(r'\.|\?|\!', synopsis) # split the title/synopsis into sentences
        
        for sentence in sentences:
            # get a set of the stopwords to remove
            stop_words = set(stopwords.words('english'))
            # for sentence in sent_tokenize(synopsis):
            # Remove non-alphabetic characters and convert to lowercase
            sentence = re.sub('[^a-zA-Z]', ' ', sentence).lower()
            # Tokenise the sentence
            sentence = word_tokenize(sentence)
            # Remove stopwords
            sentence = [word for word in sentence if word not in stop_words]
            # Lemmatize the words
            lemmatizer = WordNetLemmatizer()
            sentence = [lemmatizer.lemmatize(word) for word in sentence]
            sentence = ' '.join(sentence)
        
            cleaned_sentences.append(sentence)
    
    return cleaned_sentences


In [65]:
train_df['text'] = train_df['title'] + ' ' + train_df['plot_synopsis']
train_df.drop(columns=['title','plot_synopsis'], inplace=True)
cleaned_sentences = preprocess_and_return_sentences(list(train_df['text']))

KeyboardInterrupt: 

Call the model. This is where all the hyper parameters are set.

In [54]:
w2v = Word2Vec(cleaned_sentences, vector_size=200, window=1, min_count=1, workers=16)

Define a cosine similarity function.

This function takes in a model, and 2 words to find the cosine distance between.

We return 0 if the word is OOV (out of vocabulary).

In [56]:
def cosine_similarity(model, word_a, word_b):
    try:
        vector_a = model.wv[word_a]
    except:
        return 0
    try:
        vector_b = model.wv[word_b]
    except:
        return 0

    distance = np.dot(vector_a, vector_b) / (np.linalg.norm(vector_a) * np.linalg.norm(vector_b))
    
    return distance

print(cosine_similarity(w2v, 'bone', 'teeth'))
print(w2v.wv.similarity('bone', 'teeth'))


0


KeyError: "Key 'bone' not present"

In [40]:
data = [] 
for index, row in test_df.iterrows():
    predicted_similarity = cosine_similarity(w2v, row[1], row[2])
    data.append([row[0], predicted_similarity])
    print(row[0], row[1], row[2], predicted_similarity)
output_df = pd.DataFrame(data)
print(output_df[0])

1 absorb learn 0.21107265
2 absorb withdraw 0.2012277
3 achieve accomplish 0.6727188
4 achieve try 0.035309363
6 acquire get 0.3384742
7 acquire obtain 0.74875534
8 acquire find 0.3213722
11 apple sauce 0.3679599
12 apple lemon 0.28626597
13 apple sunshine 0.21503901
14 area region 0.4900146
16 area corner 0.3002885
17 arm shoulder 0.6949253
20 arm body 0.25594124
21 arm vein 0.43120506
22 arm knee 0.5763541
23 arm bone 0.328757
24 arm neck 0.65325624
25 ball cannon 0.2836745
26 ball basket 0.30106795
29 bath trick 0.09466636
30 bath wife 0.05536081
31 bath balloon 0.41660836
32 bed bedroom 0.64905345
33 bed blanket 0.526487
34 bed crib 0.56472844
35 bed hospital 0.31625187
38 bed chair 0.60574293
39 belief opinion 0.46524096
40 belief concept 0.59252495
41 belief impression 0.2500459
42 belief flower 0.1674661
43 bird hawk 0.33926216
44 bird turkey 0.20032915
45 bird hen 0.09921355
46 bird cock 0.25659618
48 bone jaw 0.44865564
49 bone ankle 0.36919773
50 bone knee 0.36760485
51 bone 

In [41]:
from pathlib import Path
filepath = Path('./10861383-Task1-method-b.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
print(output_df)
output_df.to_csv(filepath, index=False, header=False)  

       0         1
0      1  0.211073
1      2  0.201228
2      3  0.672719
3      4  0.035309
4      6  0.338474
..   ...       ...
145  177  0.607628
146  178  0.267023
147  179  0.312357
148  181  0.322376
149  182  0.451240

[150 rows x 2 columns]
